In [2]:
with open('./data/input_18.txt') as fh:
    file_input = fh.read().strip()

In [245]:
import networkx as nx
from networkx.algorithms.shortest_paths.generic import has_path
from networkx import NetworkXNoPath
from functools import lru_cache
from collections import OrderedDict

In [4]:
test = """########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################"""

In [5]:
test2 = """#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################"""

In [6]:
test3 = """########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################"""

In [56]:
test4 = """########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################"""

In [7]:
class L(list):
    def __new__(self, *args, **kwargs):
        return super(L, self).__new__(self, args, kwargs)

    def __init__(self, *args, **kwargs):
        if len(args) == 1 and hasattr(args[0], '__iter__'):
            list.__init__(self, args[0])
        else:
            list.__init__(self, args)
        self.__dict__.update(kwargs)

    def __call__(self, **kwargs):
        self.__dict__.update(kwargs)
        return self

In [263]:
def parse(txt):
    lines = L(txt.splitlines())
    lines.x = len(lines[0])
    lines.y = len(lines)
    return lines

def neighbors_short(pos, field):
    for x, y in ((pos[0]+1, pos[1]), (pos[0], pos[1]+1)):
        if x < field.x and x >= 0 and y < field.y and y >=0:
            yield (x, y), field[y][x] 

def neighbors_all(pos, field):
    for x, y in ((pos[0]+1, pos[1]), (pos[0], pos[1]+1), (pos[0]-1, pos[1]), (pos[0], pos[1]-1)):
        if x < field.x and x >= 0 and y < field.y and y >=0:
            yield (x, y), field[y][x]

D = tuple(range(ord('A'), ord('Z')+1))
K = tuple(range(ord('a'), ord('z')+1))
wall = ord('#')
free = ord('.')
start = ord('@')
valid_targets = (free, start) + K
diff = ord('a') - ord('A')
    
def build_graph(field):
    G = nx.Graph()
    doors = {}
    keys = {}
    home = (0, 0)
    for y, line in enumerate(field):
        for x, char in enumerate(line):
            o = ord(char)
            if o != ord('#'):
                G.add_node((x, y))
                if o in K:
                    keys[char] = (x, y)
                if o == start:
                    home = (x, y)
                if o in D:
                    doors[char] = (x, y)
                    continue
                for npos, vngb in neighbors_short((x, y), field):
                    if ord(vngb) in valid_targets:
                        G.add_edge((x, y), npos)
    
    return G, keys, doors, home

def add_door(G, pos, field):
    if pos is None:
        return G
    for npos, vngb in neighbors_all(pos, field):
        if ord(vngb) != wall:
            G.add_edge(pos, npos)
    return G

@lru_cache(maxsize=None)
def candidates(pos, leftover_keys):
    cand = []
    G_new = G.copy()
    for key in [k for k in keys if k not in leftover_keys]:
        G_new = add_door(G_new, doors.get(key.upper(), None), field)
        
    for k in leftover_keys:
        try:
            path = nx.shortest_path(G_new, pos, keys[k])
            length = len(path) - 1
            append = True
            for key in leftover_keys:
                if keys[key] in path[1:-1]:
                    append = False
                    break
            if append:
                cand.append((k, length))
        except NetworkXNoPath:
            pass
    return sorted(cand, key=lambda x: x[1])

def walk(pos, keys, doors, field):
    paths = {(pos, tuple(sorted((keys.keys())))): [0, [], set([])]}
    paths = OrderedDict(paths)
    final = []
    N = 0
    while paths:
        idx, path = paths.popitem(False)
#         print(path)
        cur, leftkeys = idx
        length, order, visited = path
        cands = candidates(cur, leftkeys)
        for key, path_len in cands:
            leftkeys_new = tuple(sorted([k for k in leftkeys if k != key]))
            if len(leftkeys_new):
                new_visited = visited | set(key)
                insert_key = (keys[key], leftkeys_new)
                if insert_key in paths:
                    if length+path_len < paths[insert_key][0]:
#                             print('replacing')
                            paths[insert_key][0] = length+path_len
                            paths[insert_key][1] = order + [key]
                else:
                    paths[insert_key] = [length+path_len, order + [key], set(order+[key])]
                    paths.move_to_end(insert_key)
#                 print(paths)
            else:
                final.append((length+path_len, order + [key]))
                
    return final

In [304]:
field = parse(test)
G, keys, doors, home = build_graph(field)

paths = walk(home, keys, doors, field)
sorted(paths)[0]

(86, ['a', 'b', 'c', 'd', 'e', 'f'])

In [262]:
field = parse(test2)
G, keys, doors, home = build_graph(field)

paths  = walk(home, keys, doors, field)
sorted(paths)[0]

(136,
 ['a',
  'f',
  'b',
  'j',
  'h',
  'd',
  'l',
  'g',
  'n',
  'o',
  'c',
  'i',
  'e',
  'p',
  'k',
  'm'])

In [127]:
field = parse(test3)
G, keys, doors, home = build_graph(field)

paths = walk(home, keys, doors, field)
sorted(paths)[0]

(81, ['a', 'c', 'd', 'g', 'f', 'i', 'b', 'e', 'h'])

In [108]:
field = parse(test4)
G, keys, doors, home = build_graph(field)
print(home)

paths = walk(home, keys, doors, field)
sorted(paths)[0]

(6, 3)


(132, ['b', 'a', 'c', 'd', 'f', 'e', 'g'])

In [264]:
field = parse(file_input)
G, keys, doors, home = build_graph(field)

paths = walk(home, keys, doors, field)
sorted(paths)[0]

(4762,
 ['i',
  'z',
  'v',
  'm',
  'f',
  'c',
  'e',
  'o',
  's',
  'p',
  't',
  'k',
  'j',
  'q',
  'w',
  'b',
  'x',
  'g',
  'l',
  'h',
  'u',
  'n',
  'r',
  'd',
  'a',
  'y'])

In [267]:
########################################################

In [266]:
# Part 2

In [372]:
def parse(txt):
    lines = L(txt.splitlines())
    lines.x = len(lines[0])
    lines.y = len(lines)
    return lines

def neighbors_short(pos, field):
    for x, y in ((pos[0]+1, pos[1]), (pos[0], pos[1]+1)):
        if x < field.x and x >= 0 and y < field.y and y >=0:
            yield (x, y), field[y][x] 

def neighbors_all(pos, field):
    for x, y in ((pos[0]+1, pos[1]), (pos[0], pos[1]+1), (pos[0]-1, pos[1]), (pos[0], pos[1]-1)):
        if x < field.x and x >= 0 and y < field.y and y >=0:
            yield (x, y), field[y][x]

D = tuple(range(ord('A'), ord('Z')+1))
K = tuple(range(ord('a'), ord('z')+1))
wall = ord('#')
free = ord('.')
start = ord('@')
valid_targets = (free, start) + K + D
diff = ord('a') - ord('A')
    
def build_graph(field):
    G = nx.Graph()
    doors = {}
    keys = {}
    home = (0, 0)
    for y, line in enumerate(field):
        for x, char in enumerate(line):
            o = ord(char)
            if o != ord('#'):
                G.add_node((x, y))
                if o in K:
                    keys[char] = (x, y)
                if o == start:
                    home = (x, y)
                if o in D:
                    doors[char] = (x, y)
                    
                for npos, vngb in neighbors_short((x, y), field):
                    if ord(vngb) in valid_targets:
                        G.add_edge((x, y), npos)
    
    return G, keys, doors, home

def add_door(G, pos, field):
    if pos is None:
        return G
    for npos, vngb in neighbors_all(pos, field):
        if ord(vngb) != wall:
            G.add_edge(pos, npos)
    return G

@lru_cache(maxsize=None)
def candidates(i, pos, leftover_keys):
    cand = []
    G_new = G[i].copy()
    for key in doors[i].keys():
        G_new = add_door(G_new, doors[i].get(key.upper(), None), f[i])
        
    for k in leftover_keys:
        try:
            path = nx.shortest_path(G_new, pos, keys[i][k])
            length = len(path) - 1
            append = True
            for key in leftover_keys:
                if keys[i][key] in path[1:-1]:
                    append = False
                    break
            if append:
                cand.append((k, length))
        except NetworkXNoPath:
            pass
    return sorted(cand, key=lambda x: x[1])

def walk(pos, keys, doors, field):
    paths = {
        (pos[i], i, tuple(sorted((keys[i].keys())))): 
            [0, [], set([])] for i in range(4)
    }
    paths = OrderedDict(paths)
    final = []
    N = 0
    while paths:
        idx, path = paths.popitem(False)
#         print(path)
        cur, sub, leftkeys = idx
        length, order, visited = path
        cands = candidates(sub, cur, leftkeys)
        for key, path_len in cands:
            leftkeys_new = tuple(sorted([k for k in leftkeys if k != key]))
            if len(leftkeys_new):
                new_visited = visited | set(key)
                insert_key = (keys[sub][key], sub, leftkeys_new)
                if insert_key in paths:
                    if length+path_len < paths[insert_key][0]:
#                             print('replacing')
                            paths[insert_key][0] = length+path_len
                            paths[insert_key][1] = order + [key]
                else:
                    paths[insert_key] = [length+path_len, order + [key], set(order+[key])]
                    paths.move_to_end(insert_key)
#                 print(paths)
            else:
                final.append((length+path_len, order + [key]))
                
    return final

In [373]:
field = parse(file_input)

In [374]:
f = []
f.append(L([f[:40] for f in field[:40]]))
f.append(L([f[41:] for f in field[:40]]))
f.append(L([f[:40] for f in field[41:]]))
f.append(L([f[41:] for f in field[41:]]))

In [375]:
G = []
keys = []
ikeys = []
doors = []
idoors = []
home = ((39, 39), (0, 39), (39, 0), (0, 0))
for i in range(4):
    f[i].x = 40
    f[i].y = 40
    _1, _2, _3, _4 = build_graph(f[i])
    G.append(_1)
    keys.append(_2)
    ikeys.append( {v: k for k, v in _2.items()})
    doors.append(_3)
    idoors.append( {v: k for k, v in _3.items()})


In [376]:
paths = walk(home, keys, doors, field)
sorted(paths)[0]

(300, ['c', 'e', 'j'])

In [377]:
paths

[(300, ['c', 'e', 'j']),
 (552, ['i', 'z', 'v', 'q', 'w', 'b']),
 (934, ['i', 'z', 'q', 'w', 'b', 'v']),
 (868, ['q', 'w', 'i', 'z', 'v', 'b']),
 (746, ['q', 'w', 'b', 'i', 'z', 'v']),
 (522, ['o', 's', 'p', 't', 'x', 'g', 'l', 'h']),
 (852, ['o', 'p', 't', 'x', 'g', 'l', 'h', 's']),
 (1262, ['p', 't', 'x', 'g', 'l', 'o', 's', 'h']),
 (844, ['p', 't', 'x', 'g', 'l', 'h', 'o', 's']),
 (502, ['m', 'f', 'k', 'u', 'n', 'r', 'd', 'a', 'y']),
 (730, ['m', 'f', 'u', 'n', 'r', 'd', 'a', 'y', 'k']),
 (926, ['m', 'f', 'u', 'n', 'r', 'd', 'a', 'k', 'y']),
 (806, ['f', 'k', 'u', 'n', 'r', 'd', 'a', 'y', 'm']),
 (1074, ['f', 'k', 'u', 'n', 'r', 'd', 'a', 'm', 'y']),
 (806, ['f', 'u', 'n', 'r', 'd', 'a', 'y', 'k', 'm']),
 (878, ['f', 'u', 'n', 'r', 'd', 'a', 'y', 'm', 'k'])]

In [378]:
300 + 552 + 522 + 502

1876

In [350]:
i = 0
print(keys[i])
for key in keys[i]:
    try:
        path = nx.shortest_path(G[i], keys[i][key], home[i])
        for s in path[1:-1]:
            if s in ikeys[i]:
                print('Key:', key, s, ikeys[i][s])
            if s in idoors[i]:
                print('Doors:', key, s, idoors[i][s])
    except NetworkXNoPath:
        print(key)

{'j': (23, 15), 'e': (21, 25), 'c': (21, 37)}
Doors: j (13, 10) K
Key: j (21, 25) e
Doors: j (4, 7) V
Key: j (21, 37) c
Doors: e (4, 7) V
Key: e (21, 37) c


In [356]:
i = 3
print(keys[i].keys())
for key in keys[i]:
    try:
        path = nx.shortest_path(G[i], keys[i][key], home[i])
        for s in path[1:-1]:
            if s in ikeys[i]:
                print('Key:', key, s, ikeys[i][s])
            if s in idoors[i]:
                print('Doors:', key, s, idoors[i][s])
    except NetworkXNoPath:
        print(key)

dict_keys(['w', 'i', 'q', 'v', 'z', 'b'])
Key: w (32, 22) q
Doors: w (27, 4) C
Doors: q (27, 4) C
Key: v (10, 34) z
Key: v (2, 18) i
Key: z (2, 18) i
Doors: b (18, 31) J
Doors: b (22, 15) M
Key: b (24, 12) w
Key: b (32, 22) q
Doors: b (27, 4) C
